Шапошников Максим<br>
12 место на private (0.943), 12 место на public (0.94)<br>
Подход: xgboost на выделенных фичах + результат логистической регрессии на n-grams и knn на выделенных фичах как дополнительные признаки

In [1]:
import pandas as pd
import numpy as np

In [2]:
train_data = pd.read_csv('train.csv')

## признак 1 - заглавная буква

In [3]:
train_data['First_is_upper'] = train_data.apply(lambda row: 1 if row.Word[0].isupper() and row.Word[1:].islower() else 0, axis=1)

## признак 2 - слово с маленькой буквы

In [4]:
train_data['Starts_with_lower'] = train_data.apply(lambda row: 1 if row.Word.islower() else 0, axis=1)

## признак 3 - в слове все буквы заглавные

In [5]:
train_data['All_upper'] = train_data.apply(lambda row: 1 if row.Word.isupper() else 0, axis=1)

## признак 4 - слово в CamelCase

In [6]:
def camel_case_detector(word):
    res = [x for x in word if x.isupper()]
    if len(res) >= 2 and len(res) < len(word):
        return 1
    return 0
train_data['CamelCase'] = train_data.apply(lambda row: camel_case_detector(row.Word), axis=1)

## признак 5 - количество гласных

In [7]:
import re
train_data['Count_Vowels'] = train_data.apply(lambda row: len(re.findall('[уеыаоэяию]', row.Word, re.IGNORECASE)), axis=1)

## признак 6 - наличие дополнительных символов

In [8]:
import string
def contains_special_symbols(word):
    for w in word:
        if w in string.punctuation or w == '':
            return 1
    return 0
train_data['Has_special_symbols'] = train_data.apply(lambda row: contains_special_symbols(row.Word), axis=1)

## признак 7 - Pymorphy распознал слово как фамилию

In [9]:
import pymorphy2
analyzer = pymorphy2.MorphAnalyzer()
def is_surname_by_pymorphy(word):
    parsed = analyzer.parse(word)
    for variant in parsed:
        if 'Surn' in variant.tag:
            return 1
    return 0
train_data['pymorphy'] = train_data.apply(lambda row: is_surname_by_pymorphy(row.Word), axis=1)

## Признак 8 - Есть ли дубликат из другого класса
для линйной модели критично и лучше вообще выбросить дубликаты
для деревьев может - могут сосуществовать вместе

## Признак 9 - Word2Vec модель на Фамилия + слово

In [10]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('ruscorpora_1_300_10.bin.gz', binary=True, encoding='utf-8')

C:\Users\1\Anaconda3\lib\site-packages\gensim\utils.py:1167: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [52]:
def word2vec_similarity(word):
    for_vec_model = '{}_NOUN'.format(word.lower())
    try:
        return model.similarity('фамилия_NOUN', for_vec_model)
    except KeyError as e:
        return 0
train_data['word2vecSim'] = train_data.apply(lambda row: word2vec_similarity(row.Word), axis=1)

## Признак 10 - Pymystem3 для уточнения - фамилия или нет

In [ ]:
import pymystem3
stemmer = pymystem3.Mystem()

In [ ]:
# def stemming(word):
#     response = stemmer.analyze(word)
#     try:
#         analysis = response[0].get('analysis')[0].get('gr')
#         if 'фам' in analysis:
#             return 1
#     except Exception:
#         return 0
#     return 0
# train_data['pymyStem'] = train_data.apply(lambda row: stemming(row.Word), axis=1)

## Признак 11 - Natasha 

In [12]:
predictions_from_Natasha = pd.read_csv('train_with_natasha.csv').Natasha.values
train_data['Natasha'] = predictions_from_Natasha

## Признак 12 - Длина слова

In [13]:
train_data['Length'] = train_data['Word'].apply(lambda x: len(x))

## Признак 13 - Количество согласных

In [14]:
train_data['Count_Not_Vowels'] = train_data['Length'] - train_data['Count_Vowels']

## Признак 14 - группа признаков из pyMorphy тегов

In [15]:
from sklearn.preprocessing import LabelEncoder
morph = pymorphy2.MorphAnalyzer()

train_data['pymorphyObj'] = train_data['Word'].apply(lambda x: morph.tag(x)[0])

train_data['pymorphy_animacy'] = train_data['pymorphyObj'].apply(lambda x: x.animacy)
train_data['pymorphy_POS'] = train_data['pymorphyObj'].apply(lambda x: x.POS)
train_data['pymorphy_case'] = train_data['pymorphyObj'].apply(lambda x: x.case)
train_data['pymorphy_number'] = train_data['pymorphyObj'].apply(lambda x: x.number)
train_data['pymorphy_gender'] = train_data['pymorphyObj'].apply(lambda x: x.gender)

train_data.drop('pymorphyObj' , axis=1 , inplace=True)

columns_to_one_hot = ['pymorphy_animacy', 'pymorphy_POS', 'pymorphy_case','pymorphy_number', 'pymorphy_gender']

for col in columns_to_one_hot:
    train_data[col] = LabelEncoder().fit_transform(list(train_data[col].fillna('nan')))


## Признак 14(2) - предсказания pyMystem3

In [18]:
predictions_from_stem = pd.read_csv('train_with_natashaAndPymystem.csv')
train_data['pyStemName'] = predictions_from_stem.pymystemName
train_data['pyStemSurname'] = predictions_from_stem.pymystemSurname

## Признак 15 - Результат линейной модели

In [27]:
train_data['linearModel'] = linear_preds

## Признак 16 - KNN модель

In [66]:
train_data['KnnPred'] = knnpreds

## Функция для формирования результата

In [28]:
def make_submission(predictions, filename='submit.csv'):
    submit = pd.DataFrame({'Prediction': predictions})
    submit.index.name = 'Id'
    submit.to_csv(filename)

In [29]:
train_data.head()

,Word,Label,First_is_upper,Starts_with_lower,All_upper,CamelCase,Count_Vowels,Has_special_symbols,pymorphy,word2vecSim,...,Length,Count_Not_Vowels,pymorphy_animacy,pymorphy_POS,pymorphy_case,pymorphy_number,pymorphy_gender,pyStemName,pyStemSurname,linearModel
0,Аалтонен,1,1,0,0,0,4,0,0,0.000000,...,8,4,2,1,7,2,1,0,0,0.409957
1,Аар,0,1,0,0,0,2,0,0,0.201280,...,3,1,2,17,7,0,2,0,0,0.200070
2,Аарон,0,1,0,0,0,3,0,1,0.168421,...,5,2,0,8,8,2,1,1,0,0.274166
3,ААРОН,0,0,0,1,0,3,0,1,0.168421,...,5,2,0,8,8,2,1,1,0,0.274166
4,Аарона,0,1,0,0,0,4,0,1,0.000000,...,6,2,0,8,4,2,1,0,1,0.259846


## Выделение признаков в test.csv

In [31]:
test_data = pd.read_csv('test.csv')

In [76]:
predictions_from_Natasha = pd.read_csv('test_with_natashaAndPymystem.csv')
test_data['First_is_upper'] = test_data.apply(lambda row: 1 if row.Word[0].isupper() and row.Word[1:].islower() else 0, axis=1)
test_data['Starts_with_lower'] = test_data.apply(lambda row: 1 if row.Word.islower() else 0, axis=1)
test_data['All_upper'] = test_data.apply(lambda row: 1 if row.Word.isupper() else 0, axis=1)
test_data['CamelCase'] = test_data.apply(lambda row: camel_case_detector(row.Word), axis=1)
test_data['Count_Vowels'] = test_data.apply(lambda row: len(re.findall('[уеыаоэяию]', row.Word, re.IGNORECASE)), axis=1)
test_data['Has_special_symbols'] = test_data.apply(lambda row: contains_special_symbols(row.Word), axis=1)
test_data['pymorphy'] = test_data.apply(lambda row: is_surname_by_pymorphy(row.Word), axis=1)
test_data['word2vecSim'] = test_data.apply(lambda row: word2vec_similarity(row.Word), axis=1)
test_data['Natasha'] = predictions_from_Natasha.Natasha
test_data['Length'] = test_data['Word'].apply(lambda x: len(x))
test_data['Count_Not_Vowels'] = test_data['Length'] - test_data['Count_Vowels']
test_data['pymorphyObj'] = test_data['Word'].apply(lambda x: morph.tag(x)[0])

test_data['pymorphy_animacy'] = test_data['pymorphyObj'].apply(lambda x: x.animacy)
test_data['pymorphy_POS'] = test_data['pymorphyObj'].apply(lambda x: x.POS)
test_data['pymorphy_case'] = test_data['pymorphyObj'].apply(lambda x: x.case)
test_data['pymorphy_number'] = test_data['pymorphyObj'].apply(lambda x: x.number)
test_data['pymorphy_gender'] = test_data['pymorphyObj'].apply(lambda x: x.gender)

test_data.drop('pymorphyObj' , axis=1 , inplace=True)

columns_to_one_hot = ['pymorphy_animacy', 'pymorphy_POS', 'pymorphy_case','pymorphy_number', 'pymorphy_gender']

for col in columns_to_one_hot:
    test_data[col] = LabelEncoder().fit_transform(list(test_data[col].fillna('nan')))
test_data['pyStemName'] = predictions_from_Natasha.pymystemName
test_data['pyStemSurname'] = predictions_from_Natasha.pymystemSurname
test_data['linearModel'] = linear_preds
test_data['KnnPred'] = knnpreds

## тестовая и отложенная выборки на обучении

In [71]:
from sklearn.model_selection import train_test_split
drop_features = ['Word', 'Label']
feature_names = [c for c in train_data if c not in drop_features]
X = train_data[feature_names]
y = train_data['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, shuffle=True, test_size=0.1)

## xgboost модель - 0.915

In [73]:
import xgboost as xgb
XG_train = xgb.DMatrix(X_train, label=y_train)
XG_test = xgb.DMatrix(X_test, label=y_test)
params = {'max_depth': 10, 
          'min_child_weight': 20, 
          'lambda': 0.1,
          'alpha': 0.1,
          'eta': 0.3, 
          'objective': 'binary:logistic',
          'eval_metric' : 'auc',
          'silent': 1,
          'seed': 17
         }
watchlist = [(XG_train, 'train'), (XG_test, 'valid')]
xgbmodel = xgb.train(params, XG_train, 30000, watchlist, early_stopping_rounds=100, verbose_eval=10)

[0]	train-auc:0.951898	valid-auc:0.949133
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 100 rounds.
[10]	train-auc:0.961695	valid-auc:0.95534
[20]	train-auc:0.964442	valid-auc:0.956187
[30]	train-auc:0.965821	valid-auc:0.956359
[40]	train-auc:0.966794	valid-auc:0.956468
[50]	train-auc:0.96724	valid-auc:0.956369
[60]	train-auc:0.968155	valid-auc:0.956318
[70]	train-auc:0.968787	valid-auc:0.956129
[80]	train-auc:0.969195	valid-auc:0.956335
[90]	train-auc:0.969578	valid-auc:0.956136
[100]	train-auc:0.96998	valid-auc:0.956037
[110]	train-auc:0.970337	valid-auc:0.955927
[120]	train-auc:0.970578	valid-auc:0.955918
[130]	train-auc:0.970857	valid-auc:0.955944
Stopping. Best iteration:
[36]	train-auc:0.966603	valid-auc:0.956546



In [77]:
submission_features = [c for c in test_data if c not in ['Word','Id']]
XG_submission = xgb.DMatrix(test_data[submission_features])
predictions = xgbmodel.predict(XG_submission)
make_submission(predictions, 'xgboostModel7.csv')

## xgboost о валидации

In [ ]:
train_data = train_data.drop_duplicates(subset=['Word'], keep=False)

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
drop_features = ['Word', 'Label', 'Id']
feature_names = [c for c in train_data if c not in drop_features]
X = train_data[feature_names]
y = train_data['Label']
modelXGB = XGBClassifier(max_depth = 10 , n_estimators=670 , learning_rate=0.09 , colsample_bytree=0.9 , colsample_bylevel=0.6)
cv = StratifiedKFold(4 ,shuffle=True, random_state=99)
score = cross_val_score( modelXGB, X, y, scoring='roc_auc' , cv=cv)
print (score.mean() , score.std() , '\n')

In [ ]:
modelXGB = XGBClassifier(max_depth = 10 , n_estimators=670 , learning_rate=0.09 , colsample_bytree=0.9 , colsample_bylevel=0.6)
X = train_data[feature_names]
y = train_data['Label']
modelXGB.fit(X, y)
submission_features = [c for c in test_data if c not in ['Word','Id']]
X_test = test_data[submission_features]
predictions = modelXGB.predict(X_test)
make_submission(predictions, 'xgboost_linModel.csv')

In [ ]:
X_test.shape

## Linear Model, логистическая регрессия

In [19]:
data_copy = train_data.drop_duplicates(subset=['Word'], keep=False)

In [20]:
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import GridSearchCV

vectorizer = CountVectorizer(ngram_range=(2,10), analyzer='char_wb')
skf = StratifiedKFold(random_state=42, n_splits=3)

In [21]:
x_train_vec = vectorizer.fit_transform(data_copy.Word)

In [ ]:
# from scipy.sparse import hstack
# dropped_features = ['Word', 'Label', 'Count_Not_Vowels']
# x_train_features = train_data[[c for c in train_data if c not in dropped_features]]
# X_train = hstack([x_train_vec, x_train_features])

In [22]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
param_grid = {
    'C' : [0.1, 0.01, 0.001, 1, 10, 100],
    'penalty': ['l1', 'l2'],
}
estimator = LogisticRegression()
grid = GridSearchCV(estimator=estimator, param_grid=param_grid, scoring='roc_auc', cv=skf, n_jobs=-1)
grid.fit(x_train_vec, data_copy.Label)

GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=42, shuffle=False),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'C': [0.1, 0.01, 0.001, 1, 10, 100], 'penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [23]:
grid.grid_scores_, grid.best_score_

C:\Users\1\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:747: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.66635, std: 0.12044, params: {'C': 0.1, 'penalty': 'l1'},
  mean: 0.75644, std: 0.07461, params: {'C': 0.1, 'penalty': 'l2'},
  mean: 0.66458, std: 0.05154, params: {'C': 0.01, 'penalty': 'l1'},
  mean: 0.78484, std: 0.04029, params: {'C': 0.01, 'penalty': 'l2'},
  mean: 0.50000, std: 0.00000, params: {'C': 0.001, 'penalty': 'l1'},
  mean: 0.78306, std: 0.01929, params: {'C': 0.001, 'penalty': 'l2'},
  mean: 0.65052, std: 0.13398, params: {'C': 1, 'penalty': 'l1'},
  mean: 0.74088, std: 0.08366, params: {'C': 1, 'penalty': 'l2'},
  mean: 0.64361, std: 0.12804, params: {'C': 10, 'penalty': 'l1'},
  mean: 0.73435, std: 0.08533, params: {'C': 10, 'penalty': 'l2'},
  mean: 0.63388, std: 0.13190, params: {'C': 100, 'penalty': 'l1'},
  mean: 0.73190, std: 0.08680, params: {'C': 100, 'penalty': 'l2'}],
 0.78484468851682987)

In [35]:
estim = grid.best_estimator_
x_test_vec = vectorizer.transform(test_data.Word)
linear_preds = [x[1] for x in estim.predict_proba(x_test_vec)]

In [ ]:
linearModel = grid.best_estimator_
x_test_vec = vectorizer.transform(test_data.Word)
x_test_features = test_data[[c for c in test_data if c not in ['Word']]]
X_test = hstack([x_test_vec, x_test_features])

In [ ]:
predictions = [x[1] for x in linearModel.predict_proba(X_test)]

## KNN расстояние

In [60]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
params = {
    'weights' : ['uniform', 'distance'],
    'algorithm' : ['auto', 'kd_tree'],
}
grid = GridSearchCV(estimator=knn, param_grid=params, scoring='roc_auc', cv=skf, n_jobs=-1)
feature_names = [c for c in train_data if c not in ['Word', 'Label', 'Id']]
X = train_data[feature_names]
y = train_data['Label']
grid.fit(X, y)

GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=42, shuffle=False),
       error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'weights': ['uniform', 'distance'], 'algorithm': ['auto', 'kd_tree']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [75]:
estim = grid.best_estimator_
feature_names = [c for c in test_data if c not in ['Word', 'Label', 'Id']]
X = test_data[feature_names]
knnpreds = estim.predict(X)

In [ ]:
make_submission(predictions, 'logisticRegression.csv')